In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
import tensorflow as tf

stemmer = SnowballStemmer('english')

df_train = pd.read_csv('train.csv', encoding="ISO-8859-1") #update here 74067
df_test = pd.read_csv('test.csv', encoding="ISO-8859-1") #update here 166693
df_pro_desc = pd.read_csv('product_descriptions.csv')[:64000] #update here 64000
df_attr = pd.read_csv('attributes.csv')
num_train = df_train.shape[0]

In [2]:
#def str_stemmer(s):
	#return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
	return sum(int(str2.find(word)>=0) for word in str1.split())

In [3]:
import re
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}
def str_stemmer(s): 
    if isinstance(s,str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        #s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        return s
    else:
        return "null"

In [4]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
df_brand[0:9]

,product_uid,brand
9,100001.0,Simpson Strong-Tie
37,100002.0,BEHR Premium Textured DeckOver
69,100003.0,STERLING
93,100004.0,Grape Solar
122,100005.0,Delta
163,100006.0,Whirlpool
204,100007.0,Lithonia Lighting
236,100008.0,Teks
256,100009.0,House of Fara


In [5]:
collection = list(df_attr.name)
import re
from fuzzyfinder import fuzzyfinder
#suggestions = fuzzyfinder('material', L)
def fuzzyfinder(user_input, collection):
        suggestions =[]
        pattern = '.*'.join(user_input) # Converts 'djm' to 'd.*j.*m'
        regex = re.compile(pattern)     # Compiles a regex.
        for item in collection:
            match = regex.search(str(item))  # Checks if the current item matches the regex.
            if match:
                #suggestions.append((match.start(), item))
                suggestions.append(item)
        return suggestions
material_lower = fuzzyfinder('material', collection)
material_upper = fuzzyfinder('Material', collection)
material_lower.extend(material_upper)

df_materials = df_attr[df_attr['name'].isin(material_lower)][["product_uid", "value"]].rename(columns={"value": "material"})
df_materials=df_materials.drop_duplicates(subset=['product_uid'], keep='first')
df_materials[1:5]

,product_uid,material
67,100003.0,Composite
202,100007.0,Plastic
229,100008.0,Steel
255,100009.0,Medium Density Fiberboard (MDF)


In [6]:
df_type = fuzzyfinder('Type', collection)
df_type_= fuzzyfinder('type', collection)
df_type.extend(df_type_)
df_types = df_attr[df_attr['name'].isin(df_type)][["product_uid", "name"]].rename(columns={"name": "type"})
df_types=df_types.drop_duplicates(subset=['product_uid'], keep='first')
df_types[1:5]

,product_uid,type
66,100003.0,Installation Type
90,100004.0,Electrical Product Type
107,100005.0,Bath Faucet Type
132,100006.0,Appliance Type


In [7]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True) #train+test zongxiangdiejia 240760
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid') #240760
df_all = pd.merge(df_all, df_materials, how='left', on='product_uid')

df_all = pd.merge(df_all, df_types, how='left', on='product_uid')
df_all[0:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,material,type
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,Galvanized Steel,NaN
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,Galvanized Steel,NaN
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver,NaN,Paint Product Type
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...,Delta,NaN,Bath Faucet Type
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...,Delta,NaN,Bath Faucet Type


In [8]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stemmer(x))
df_all['material'] = df_all['material'].map(lambda x:str_stemmer(x))
df_all['type'] = df_all['type'].map(lambda x:str_stemmer(x))

In [9]:
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)

df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

In [10]:
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

In [11]:
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
#df_all['attr_brand'] = df_all['search_term']+"\t"+df_all['brand']
#df_all['attr_material'] = df_all['search_term']+"\t"+df_all['material']
#df_all['attr_function'] = df_all['search_term']+"\t"+df_all['function_description']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']+"\t"+df_all['material']+"\t"+"\t"+df_all['type']
df_all[0:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,material,type,len_of_query,product_info,word_in_title,word_in_description,query_in_title,query_in_description,query_last_word_in_title,query_last_word_in_description,ratio_title,ratio_description,attr
0,2,simpson strong tie 12 gaug angl,100001,3.00,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,galvan steel,null,2,angl bracket\tsimpson strong tie 12 gaug angl\...,1,1,1,1,0,0,0.500000,0.500000,angl bracket\tsimpson strong tie\tgalvan steel...
1,3,simpson strong tie 12 gaug angl,100001,2.50,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,galvan steel,null,2,l bracket\tsimpson strong tie 12 gaug angl\tno...,1,1,1,1,0,0,0.500000,0.500000,l bracket\tsimpson strong tie\tgalvan steel\t\...
2,9,behr premium textur deckov 1 gal #sc 141 tugbo...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,null,paint product type,2,deck over\tbehr premium textur deckov 1 gal #s...,1,1,1,1,0,0,0.500000,0.500000,deck over\tbehr premium textur deckov\tnull\t\...
3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,delta,null,bath faucet type,3,rain shower head\tdelta vero 1 handl shower on...,1,1,1,1,0,0,0.333333,0.333333,rain shower head\tdelta\tnull\t\tbath faucet type
4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,delta,null,bath faucet type,3,shower onli faucet\tdelta vero 1 handl shower ...,3,3,3,3,1,1,1.000000,1.000000,shower onli faucet\tdelta\tnull\t\tbath faucet...


In [12]:
df_all = df_all.drop(['search_term','product_title','product_description','product_info','brand','material','type','attr'],axis=1)
df_all[1:3]

,id,product_uid,relevance,len_of_query,word_in_title,word_in_description,query_in_title,query_in_description,query_last_word_in_title,query_last_word_in_description,ratio_title,ratio_description
1,3,100001,2.5,2,1,1,1,1,0,0,0.5,0.5
2,9,100002,3.0,2,1,1,1,1,0,0,0.5,0.5


In [13]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

#y_train = df_train['relevance'].values
#X_train = df_train.drop(['id','relevance'],axis=1).values
#X_test = df_test.drop(['id','relevance'],axis=1).values

In [14]:
#Here start the neural network algorithm!!!

In [15]:
import tensorflow.contrib.layers as layers
import tensorflow.contrib.losses as losses
import tensorflow.contrib.metrics as metrics
from sklearn.cross_validation import train_test_split


/Users/Helen/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
df_train = df_train.drop(axis=1, labels=['product_uid'])
df_test = df_test.drop(axis=1, labels=['product_uid'])
df_train

,id,relevance,len_of_query,word_in_title,word_in_description,query_in_title,query_in_description,query_last_word_in_title,query_last_word_in_description,ratio_title,ratio_description
0,2,3.00,2,1,1,1,1,0,0,0.500000,0.500000
1,3,2.50,2,1,1,1,1,0,0,0.500000,0.500000
2,9,3.00,2,1,1,1,1,0,0,0.500000,0.500000
3,16,2.33,3,1,1,1,1,0,0,0.333333,0.333333
4,17,2.67,3,3,3,3,3,1,1,1.000000,1.000000
5,18,3.00,2,1,2,1,2,0,1,0.500000,1.000000
6,20,2.67,3,2,2,2,2,0,0,0.666667,0.666667
7,21,3.00,1,1,1,1,1,1,1,1.000000,1.000000
8,23,2.67,2,2,2,2,2,1,1,1.000000,1.000000
9,27,3.00,2,2,2,2,2,1,1,1.000000,1.000000


In [17]:
#Set the train and test set

Y_train = df_train['relevance']
X_train = df_train.drop(axis=1, labels=['id','relevance'])

X_test = df_test.drop(axis=1, labels=['id','relevance'])

In [18]:
#Do the train

featurenum=X_train.shape[1]
rate = 0.001

x = tf.placeholder(tf.float32, shape=(None, featurenum))
y = tf.placeholder(tf.float32, shape=(None,1))
prob = tf.placeholder(tf.float32)

logits = layers.fully_connected(x, 10, activation_fn=tf.nn.relu)
logits = layers.dropout(logits, keep_prob=prob)
logits = layers.fully_connected(x, 10, activation_fn=tf.nn.relu)
logits = layers.dropout(logits, keep_prob=prob)
y_predict = layers.fully_connected(logits, 1)

loss = losses.mean_squared_error(y, y_predict)

train_optimize = tf.train.AdamOptimizer(rate).minimize(loss)

Instructions for updating:
Use tf.losses.mean_squared_error instead.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.


In [19]:
#Set batch and do the training

Total_number = X_train.shape[0]
Batch_size = 20
n_epoch = 5
n_batch = int(Total_number / Batch_size)
print("Feeding {} batches per epoch".format(n_batch))
begin = 0

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

for _ in range(n_epoch):
    begin = 0
    for batch_ind in range(n_batch-1):
    #for batch_idx in range(15):
        feeding_dict = { x: X_train.iloc[begin:(begin+Batch_size)].values,
                         y: Y_train.iloc[begin:(begin+Batch_size)].values.reshape(-1, 1),
                         prob:0.5
                       }
        begin = begin+Batch_size

        _, l  = sess.run([train_optimize, loss], feed_dict=feeding_dict)

        if not(batch_ind%1000):
            print("Loss on batch {}: {}".format(batch_ind, l))

Feeding 3703 batches per epoch
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Loss on batch 0: 9.971362113952637
Loss on batch 1000: 2.05047869682312
Loss on batch 2000: 0.9986880421638489
Loss on batch 3000: 0.1773410588502884
Loss on batch 0: 0.38331595063209534
Loss on batch 1000: 0.4097120761871338
Loss on batch 2000: 0.17538245022296906
Loss on batch 3000: 0.14656314253807068
Loss on batch 0: 0.3400639593601227
Loss on batch 1000: 0.3774486184120178
Loss on batch 2000: 0.18661648035049438
Loss on batch 3000: 0.1603289544582367
Loss on batch 0: 0.3298567831516266
Loss on batch 1000: 0.35919612646102905
Loss on batch 2000: 0.18825241923332214
Loss on batch 3000: 0.18355336785316467
Loss on batch 0: 0.3012780547142029
Loss on batch 1000: 0.490090936422348
Loss on batch 2000: 0.18029843270778656
Loss on batch 3000: 0.19792568683624268


In [20]:
#Do the predict

feeding_dict = { x: X_test.values, prob:1}
y_pred = sess.run(y_predict, feed_dict=feeding_dict)
y_pred


array([[ 2.08427453],
       [ 2.14609814],
       [ 2.14609814],
       ..., 
       [ 2.32979965],
       [ 2.36656141],
       [ 2.28470683]], dtype=float32)

In [21]:
#pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)

In [22]:
# Using y_pred to do the error analysis

In [23]:
df_sol = pd.read_csv('solution.csv', encoding="ISO-8859-1")
df_sol['pred'] = y_pred
df_fliter = df_sol[df_sol.Usage == "Public"]

In [24]:
from sklearn.metrics import mean_squared_error, make_scorer
mse = mean_squared_error(df_fliter.relevance, df_fliter.pred, sample_weight=None, multioutput='uniform_average')
np.sqrt(mse)

0.50644969673479256

In [25]:
from sklearn.metrics import r2_score
R2 = r2_score(df_fliter.relevance, df_fliter.pred)  
print(R2)

0.105845793115
